In [1]:
import boto3
import pandas as pd  # Si planeas usar datos tabulares
import os
import seaborn as sns
import numpy as np
import datetime as dt
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import MinMaxScaler
from io import StringIO
from prophet import Prophet
import plotly.express as px  # For high-level data visualizations
import plotly.graph_objects as go 

In [2]:
bucket_name = 'bucketsageindex'

# Crear un cliente de S3
s3 = boto3.client('s3')

# Listar objetos en el bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Leer archivos específicos en DataFrames
df1 = None
df2 = None

for obj in response.get('Contents', []):
    print(obj["Key"])
    if obj["Key"] == "migraciones_2024.csv":
        # Descargar el archivo de S3 y leerlo con pandas
        obj_data = s3.get_object(Bucket=bucket_name, Key=obj["Key"])
        df1 = pd.read_csv(StringIO(obj_data['Body'].read().decode('utf-8')))
    elif obj["Key"] == "migraciones_2021.csv":
        # Descargar el archivo de S3 y leerlo con pandas
        obj_data = s3.get_object(Bucket=bucket_name, Key=obj["Key"])
        df2 = pd.read_csv(StringIO(obj_data['Body'].read().decode('utf-8')))

VULNRB_IPMxMZ/VULNRB_IPMxMZ.cpg
VULNRB_IPMxMZ/VULNRB_IPMxMZ.dbf
VULNRB_IPMxMZ/VULNRB_IPMxMZ.prj
VULNRB_IPMxMZ/VULNRB_IPMxMZ.shp
VULNRB_IPMxMZ/VULNRB_IPMxMZ.shx
completo_migrantes.csv
data.json
forecasting-deepar-2024-11-11-06-55-53-367/debug-output/training_job_end.ts
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/framework/training_job_end.ts
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/incremental/2024111106/1731308160.algo-1.json
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/incremental/2024111106/1731308220.algo-1.json
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/incremental/2024111106/1731308280.algo-1.json
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/incremental/2024111106/1731308340.algo-1.json
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/incremental/2024111107/1731308400.algo-1.json
forecasting-deepar-2024-11-11-06-55-53-367/profiler-output/system/training_job_end

In [3]:
all_df = pd.concat([df1, df2])

In [4]:
all_df.head()

,Unnamed: 0,cantidad_de_filas,agnio,region_destino,region_nacionalidad,entrada_salida,sexo,categoria_migratoria,motivo_viaje,departamento_hospedaje,...,colombiano_extranjero,puesto_migratorio,tipo_transporte,centro_regional,pais_nacionalidad,meses,pais_destino_procedencia,rango_edad,departamento,tipo_viajero
0,11,1,2022,América Central y el Caribe,América Central y el Caribe,Entradas,Masculino,Turismo,Turismo,Antioquia,...,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,El Salvador,Junio,El Salvador,70 o Mas,Bogotá,Visitante
1,15,3,2022,Europa,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,...,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,Noruega,Enero,Noruega,30-39,Bogotá,Visitante
2,16,33,2022,América del Sur,América del Sur,Entradas,Masculino,Permiso Otras Actividades,Tránsito,Antioquia,...,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Perú,Mayo,Perú,18-29,Antioquía,Visitante
3,75,1,2022,América del Sur,Europa,Entradas,Femenino,Visa Migrante,Vínculo Marital,Antioquia,...,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Alemania,Febrero,Chile,18-29,Antioquía,Otro viajero
4,102,1,2022,América del Norte,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,...,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Albania,Noviembre,Estados Unidos,30-39,Antioquía,Visitante


In [5]:
all_df.drop("Unnamed: 0", axis = 1, inplace = True)

In [6]:
all_df.head()

,cantidad_de_filas,agnio,region_destino,region_nacionalidad,entrada_salida,sexo,categoria_migratoria,motivo_viaje,departamento_hospedaje,ciudad_hospedaje,colombiano_extranjero,puesto_migratorio,tipo_transporte,centro_regional,pais_nacionalidad,meses,pais_destino_procedencia,rango_edad,departamento,tipo_viajero
0,1,2022,América Central y el Caribe,América Central y el Caribe,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,El Salvador,Junio,El Salvador,70 o Mas,Bogotá,Visitante
1,3,2022,Europa,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,Noruega,Enero,Noruega,30-39,Bogotá,Visitante
2,33,2022,América del Sur,América del Sur,Entradas,Masculino,Permiso Otras Actividades,Tránsito,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Perú,Mayo,Perú,18-29,Antioquía,Visitante
3,1,2022,América del Sur,Europa,Entradas,Femenino,Visa Migrante,Vínculo Marital,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Alemania,Febrero,Chile,18-29,Antioquía,Otro viajero
4,1,2022,América del Norte,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Albania,Noviembre,Estados Unidos,30-39,Antioquía,Visitante


In [7]:
fechas = all_df.reset_index()

In [8]:
fechas = fechas.drop("index", axis = 1)

In [9]:
fechas.head()

,cantidad_de_filas,agnio,region_destino,region_nacionalidad,entrada_salida,sexo,categoria_migratoria,motivo_viaje,departamento_hospedaje,ciudad_hospedaje,colombiano_extranjero,puesto_migratorio,tipo_transporte,centro_regional,pais_nacionalidad,meses,pais_destino_procedencia,rango_edad,departamento,tipo_viajero
0,1,2022,América Central y el Caribe,América Central y el Caribe,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,El Salvador,Junio,El Salvador,70 o Mas,Bogotá,Visitante
1,3,2022,Europa,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto Eldorado de Bogotá,Aéreo,Aeropuerto Eldorado,Noruega,Enero,Noruega,30-39,Bogotá,Visitante
2,33,2022,América del Sur,América del Sur,Entradas,Masculino,Permiso Otras Actividades,Tránsito,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Perú,Mayo,Perú,18-29,Antioquía,Visitante
3,1,2022,América del Sur,Europa,Entradas,Femenino,Visa Migrante,Vínculo Marital,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Alemania,Febrero,Chile,18-29,Antioquía,Otro viajero
4,1,2022,América del Norte,Europa,Entradas,Masculino,Turismo,Turismo,Antioquia,Medellín,Extranjeros,Aeropuerto José María Córdova de Rionegro,Aéreo,Antioquia,Albania,Noviembre,Estados Unidos,30-39,Antioquía,Visitante


In [10]:
fechas = fechas[["agnio", "meses"]]

In [11]:
fechas.head()

,agnio,meses
0,2022,Junio
1,2022,Enero
2,2022,Mayo
3,2022,Febrero
4,2022,Noviembre


In [12]:
formato = fechas.agnio.astype(str) + '-' + fechas.meses.astype(str)

In [13]:
fechas["DS"] = formato

In [14]:
fechas

,agnio,meses,DS
0,2022,Junio,2022-Junio
1,2022,Enero,2022-Enero
2,2022,Mayo,2022-Mayo
3,2022,Febrero,2022-Febrero
4,2022,Noviembre,2022-Noviembre
...,...,...,...
648089,2021,Febrero,2021-Febrero
648090,2021,Febrero,2021-Febrero
648091,2021,Febrero,2021-Febrero
648092,2021,Febrero,2021-Febrero


In [15]:
agrupado = fechas.groupby("DS").count()

In [16]:
datos = agrupado.reset_index()[["DS", "agnio"]]

In [17]:
datos.head()

,DS,agnio
0,2018-Abril,15919
1,2018-Agosto,17648
2,2018-Diciembre,16010
3,2018-Enero,16511
4,2018-Febrero,14733


In [18]:
month_map = {
    "Enero": "01",
    "Febrero": "02",
    "Marzo": "03",
    "Abril": "04",
    "Mayo": "05",
    "Junio": "06",
    "Julio": "07",
    "Agosto": "08",
    "Septiembre": "09",
    "Octubre": "10",
    "Noviembre": "11",
    "Diciembre": "12"
}

# Conversión de la columna DS al formato deseado
datos['DS'] = datos['DS'].str.split('-').apply(lambda x: f"{x[0]}-{month_map[x[1]]}-01")
datos['DS'] = pd.to_datetime(datos['DS'], format="%Y-%m-%d")

In [19]:
datos

,DS,agnio
0,2018-04-01,15919
1,2018-08-01,17648
2,2018-12-01,16010
3,2018-01-01,16511
4,2018-02-01,14733
...,...,...
76,2024-07-01,10791
77,2024-06-01,9485
78,2024-03-01,9546
79,2024-05-01,9400


In [20]:
datos = datos.rename(columns={"DS": "ds", "agnio": "y"})

# Crear el modelo de Prophet y ajustarlo
m = Prophet()
m.fit(datos)

07:46:02 - cmdstanpy - INFO - Chain [1] start processing
07:46:03 - cmdstanpy - INFO - Chain [1] done processing


In [21]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
441,2025-08-28
442,2025-08-29
443,2025-08-30
444,2025-08-31
445,2025-09-01


In [22]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
441,2025-08-28,-7332.552922,-12985.001679,-1687.115258
442,2025-08-29,-4565.971880,-10141.515980,704.834016
443,2025-08-30,-1521.019394,-6768.507258,3726.344876
444,2025-08-31,1726.203958,-3575.693979,7131.480054
445,2025-09-01,5093.600648,-300.161583,10616.572566


In [ ]:
m = Prophet(growth='linear', yearly_seasonality=True)
m.add_seasonality(name='yearly', period=365.25, fourier_order=10)
m.fit(datos)

# Agregar límite inferior a la cantidad de turistas
forecast = m.predict(future)
forecast['yhat'] = forecast['yhat'].clip(lower=0)

07:46:03 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:
forecast

In [ ]:
# Configurar y entrenar el modelo
m = Prophet(growth='linear', yearly_seasonality=True)
m.add_seasonality(name='yearly', period=365.25, fourier_order=20)
m.fit(datos)

# Crear el dataframe futuro
future = m.make_future_dataframe(periods=100)  # Reemplaza PERIODOS con el número de días a predecir

# Realizar la predicción
forecast = m.predict(future)

# Aplicar límite inferior a las predicciones (si es necesario)
forecast['yhat'] = forecast['yhat'].clip(lower=0)

In [ ]:
# Generar el gráfico interactivo
fig = plot_plotly(m, forecast)

# Personalizar el gráfico (opcional)
fig.update_layout(
    title='Pronóstico interactivo de cantidad de gente',
    xaxis_title='Fecha',
    yaxis_title='Cantidad de gente',
    legend_title='Leyenda',
    hovermode='x'
)

# Mostrar el gráfico
fig.show()

In [ ]:
fig2 = m.plot_components(forecast)
plt.show()